## Section: Customizing the simulation track and objects

### Exercise 1 - Move the sim app to the working folder

In [ ]:
# # Clean the build directory if present
!python3 sim_app_bundler.py --clean

# # Untar the simapp bundle
!python3 sim_app_bundler.py --untar ../deepracer-simapp.tar.gz

# # Now modify the simapp from build directory and run this command.

# # Most of the simapp files can be found here (Robomaker changes)
# # bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/
# # bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment/
# # bundle/opt/install/deepracer_simulation_environment/lib/deepracer_simulation_environment/

# # # Copying the notebook src/markov changes to the simapp (For sagemaker container)
# !rsync -av ./src/markov/ ./build/simapp/bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/markov

# !python3 sim_app_bundler.py --tar/bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/markov

# !python3 sim_app_bundler.py --tar

## Section: Customizing the simulation track and objects


You may be interested in changing tracks (adding a new track, or editing an existing one), adding objects to your track, or adding bot cars to train against.

![changetracks](img/Changetracks.png)


### Exercise 2 - Adding new tracks to your simulation application

Racing tracks with various shapes and textures are included among the 3D assets `build/simapp/bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment`.

To add a new track, you need to provide the following files:

- 3D assets `meshes/mytrack/mytrack.dae` and textures `meshes/mytrack/textures`. Note that AWS RoboMaker currently uses Gazebo to create the simulation environment, so everything needs to be compatible with Gazebo. For the already existing tracks, we provide Blender files for modifications as needed.
- `models/mytrack/mytrack.sdf` and `models/mytrack/model.config`, which point to the meshes and allows us to construct a completed model of the meshes where we specify which meshes should be visual and which should provide collision to our car 
- `worlds/myworld.world`, which takes our track model and wraps it in a world file, complete with sky and orientation.
- `routes/mytrack.npy`, contains the waypoints of our track, outside border, center, and inside border, which our simulation will use to determine things such as is the agent on the track, and how much prrogress has been made etc.


### Exercise 3 - Adding new objects (excluding bot car) on the track

To add objects, you can use the following files:

- `models/racecar_box/model.sdf` and `models/racecar_box/model.config`, contain model of a box object. Any other Gazebo compatiable object can be added by creating a folder with these files under `models` folder.
- `worlds/myworld_with_objects.world`, which takes our prior track assets and our new object models. Any number of objects can be added by including the following lines in the world file: 

```
   <include>
    <pose>0 0 0 0 0 0</pose>
    <static>true</static>
    <uri>model://models/racecar_box</uri>
    <name>box1</name>
   </include>
```
- To replace any object during training, make a note of the object name, e.g., box1, and in your environment file, import `rospy` library and `set_model_state` function.


### Exercise 4 - Adding new bot cars

A bot car is simply another robocar without the neural network, hence, uses the same assets as the robocar in   `build/simapp/bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment`. The bot car uses the same functionality such as reset in `build/simapp/bundle/opt/install/deepracer_simulation_environment/lib/deepracer_simulation_environment/car_node.py`

To add, remove, or modify the speed and lane changing behavior bot cars, use function in `src/markov/environments/deepracer_racetrack_env.py`:
- `class BotCarController` defines the functionality for the bot car behavior, including which segments on the track to spawn the bot car. Note that learning passing around corners is difficult due to partial observability, hence, initially, you can train a model to pass only on the straight segments to reduce the training time.
- `class DeepRacerRacetrackEnv(gym.Env) def __init__(self)` is used to set the number of bot cars, their speed, and lane changning frequency.


### Exercise 5 - Adding and configuring new sensors

A new sensor such as camera or LIDAR with existing ROS plug-in can be added in directly in the Gazebo car asset files located in `build/simapp/bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment/urdf/`

To add another camera: 

- `racecar.gazebo` in the original time-trial already includes a camera, to add another camerara you can replicate that code snippet with a new name. Alternatively, you can add a less to the existing camera.

- `racecar.xacro` contains the parameters to set the location of the cameras, see `_leftcam` and `_rightcam`.

- `macro.xacro` contains the joints that the sensors are attached to including cameras and lenses for the cameras. The angle of the camera lenses are adjusted in this file.


To add a LIDAR:

- `racecar.gazebo` uses a LIDAR plugin compatible with the LIDAR on the device, similar to the camera module. There are two parameters that can be customized for the LIDAR, `scan` and `range`. These parameters determine the angle bracket and range to detect light reflections. The default settings below are for +/- 1.0472 radians (60 degrees) for the scanning angle and is between 0.15 and 0.5 meters for the observability range.
```
<scan>
  <horizontal>
    <samples>64</samples>
    <resolution>1</resolution>
    <min_angle>-1.0472</min_angle>
    <max_angle>1.0472</max_angle>
  </horizontal>
</scan>
<range>
  <min>0.15</min>
  <max>0.5</max>
  <resolution>0.01</resolution>
</range>
```

- `racecar.xacro` contains the parameters to set the location of the LIDAR, similar to the camera.

- `macro.xacro` contains the joints that the sensors are attached to and its angle.




### Exercise 6 - Preparing the AWS RoboMaker Bundle

After making changes to the simulation application assets, re-bundle it using the Python file sim_app_bundler.py. We will upload the tar.gz file to the AWS RoboMaker arn later in the notebook.

The compression may take longer depending on the instance type of your Amazon SageMaker notebook.

In [ ]:
# # Most of the simapp files can be found here (Robomaker changes)
# # bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/
# # bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment/
# # bundle/opt/install/deepracer_simulation_environment/lib/deepracer_simulation_environment/

# # # Copying the notebook src/markov changes to the simapp (For sagemaker container)
 !rsync -av ./src/markov/ ./build/simapp/bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/markov

 !python3 sim_app_bundler.py --tar /bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/markov

 !python3 sim_app_bundler.py --tar

## Section: Modify the sensor combinations
### Exercise 1 - Go from single camera, to stereo camera and LIDAR


Note that we have already added all the sensor combinations into the simulation application, and thus you only have to specify the preset that you want to use. Similarly, you can go add entirely new sensors into your simulation application, add them to the car model, and then use them during training. You will have to see what makes sense from a reward fucntion point of view.

Possible sensors
 - Single camera
 - Stereo camera
 - Single camera + LIDAR
 - Stereo Camera + LIDAR
 
When you specify a new sensor configuration this inpacts the state data that will feed into the neural network.

![networkinput](img/networkinput.png)

In specifying a single image, the process is straight forward, we take 160x120 pixel image and feed that as input.

However, when you add a second sensor you need to decide if you are concatenating the inputs before feeding them into the neural network, or if you are creating a second input (double headed so to speak).

Note that each input has its own embedder pipeline. For example, converting the color image to 8 bit grayscale. You could add more steps into this embedder process.

![inputembedder](img/inputembedder.png)


To add configure stereo camera and LIDAR, you need to update
- the observations to include the new sensors added in use function in `src/markov/environments/deepracer_racetrack_env.py` and use the below code segment in `class DeepRacerRacetrackEnv(gym.Env) def __init__(self)` to define the observation space with dictionary keys corresponding to the different type of sensors:
```
self.observation_space = spaces.Dict({
        'STEREO_CAMERAS': spaces.Box(low=0, high=255,
                                shape=(TRAINING_IMAGE_SIZE[1], TRAINING_IMAGE_SIZE[0], 2),
                                dtype=np.uint8),
        'LIDAR': spaces.Box(low=0.15, high=1.0, shape=(64,))
})
```
- the input header names for the neural network in `src/markov/presets/preset.py` and the neural network architecture which are described in the next section.



## Section: Update the neural network architecture


We have partnered with Intel and use [Coach](https://nervanasystems.github.io/coach/) as reinforcement learning framework. Furthermore, we are using Tensorflow as our deep learning framework. 

The neural network architecture typically includes an input embedder, middleware, and an output head, see descriptions [here](https://nervanasystems.github.io/coach/design/network.html). In this section we are interested in changing the middleware.

Here are the Coach layer names

![coachnames](img/coachnames.png)

### Exercise 1 - Update Middleware
- Add another convolutional layer
- Add mode convolutions
- Test drop out

To change the neural network architecture, edit the preset file in `src/markov/presets/`. We provide two example preset files:
- The default neural network architecture, `src/markov/presets/default.py`, has an input embedder with a 3 layer Convolutional Neural Network (CNN).
- The attention neural network architecture, `src/markov/presets/preset_attention_layer.py`, provides an example on how to use custom layers.

Example:

```
agent_params.network_wrappers['main'].input_embedders_parameters['observation'].scheme = [Conv2d(32, 8, 4),Conv2d(64, 4, 2),Conv2dWithAttention(64, 3, 1, 256)]

```

In this example we have three convolutional layers on top of each other. The first one has 32 convolutional filters, a kernel size of 8x8, and a stride length of 4.

The source code explanation of Conv2d is [here](https://github.com/NervanaSystems/coach/blob/19ad2d60a7022bb5125855c029f27d86aaa46d64/rl_coach/architectures/tensorflow_components/layers.py)

Here are the default Coach layer presets

![coachlayerpresets](img/coachlayerpresets.png)




### Exercise 2 - Configure the  RL algorithm hyperparameters

We use Clipped PPO (as provided by [Coach](https://nervanasystems.github.io/coach/components/agents/policy_optimization/cppo.html)) as our reinformcent learning algorithm to train our network. To edit the hyperparameters of the Clipped PPO RL agent, edit the preset file in `src/markov/presets/`. The configurable hyperparameters include learning_rate, neural network structure, batch_size, discount factor. These really are vital to getting good convergence, or none at all.

## Other interesting manipulations to take note of

### Changing the training algorithm
To change the RL agent algorithm, refer to the DQN example for AWS DeepRacer. There are multiple files that need to be editted including the preset file in `src/markov/presets/`. 

### Configure the environment file with custom simulation variables

We use an environment file, `src/markov/environment/deepracer_racetrack_env.py`, which contains "step" and "reset" functions and ability to exchange messages with the Gazebo based AWS RoboMaker simulator. This environment file is shared between Amazon Sagemaker and AWS RoboMaker jobs. The environment variable - `NODE_TYPE` defines which node the code is running on. So, the expressions that have `rospy` dependencies are executed on RoboMaker only. 

### How to add noise to observations?
You can add noise to robocar camera observations by using OpenCV or other image editting packages available in Python. Note that these libraries need to be located in or copied to `build/simapp/bundle/usr/local/lib/python3.5/dist-packages` for the AWS RoboMaker to import them.

As an example, we provide a modified environment `src/markov/environment/deepracer_racetrack_env_cv2.py`, to use `cv2` and add Gaussian noise to robocar observations in `def set_next_state():` (see Lines ~241-254)

### How to add noise to actions, i.e., steering and speed, for robustness?
Adding noise to your actions also increases the robustness of your model to steady-state or tracking errors of the robocar controllers for steering and speed in the real world. Since we use discrete action, we need to add noise to their associated mappings in `class DeepRacerRacetrackCustomActionSpaceEnv(DeepRacerRacetrackEnv):` (see Lines ~575-579)


In [ ]:
# # replace the environment file with your modified version
# !cp src/markov/environments/deepracer_racetrack_env_cv2.py src/markov/environments/deepracer_racetrack_env.py

### Exercise 3 - Configure the reward function

To customize reward functions, modify `reward_function` in `src/markov/rewards/`. Note that the parameters exposed to the reward function are coded in the environment file. To create new variables for the reward function, edit the environment file, `src/markov/environment/deepracer_racetrack_env.py`.


In [ ]:
# # Reward function
# !pygmentize src/markov/rewards/default.py

### Exercise 4 - Configure the action space

Action space and steering angles can be changed by modifying `src/markov/actions/.json` file. The default action space for our RL agent is discrete, therefore, the number of actions correspond to the number of output nodes of the policy network.


In [ ]:
# # Action space
# !pygmentize src/markov/actions/model_metadata_10_state.json

### Exercise 5 - Copy custom files to S3 bucket so that Amazon SageMaker and AWS RoboMaker can pick them up

**Very important**, remember to copy the edited files from ./src/ back into S3 where SageMaker and RoboMaker will pick them up

In [ ]:
s3_location = "s3://%s/%s" % (s3_bucket, s3_prefix)
print(s3_location)

# Clean up the previously uploaded files
!aws s3 rm --recursive {s3_location}

# Make any changes to the environment and preset files below and upload these files
!aws s3 cp src/markov/environments/deepracer_racetrack_env.py {s3_location}/environments/deepracer_racetrack_env.py

!aws s3 cp src/markov/rewards/default.py {s3_location}/rewards/reward_function.py

!aws s3 cp src/markov/actions/model_metadata_10_state.json {s3_location}/model_metadata.json

!aws s3 cp src/markov/presets/default.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_attention_layer.py {s3_location}/presets/preset.py

## Starting our Amazon SageMaker Notebook DeepRacer training environment


### Imports

In [ ]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from time import gmtime, strftime
import time
from IPython.display import Markdown
from markdown_helper import *

### Initializing basic parameters

**Call-out, please check Amazon SageMaker instance type** - it has a cost implication


In [ ]:
# Select the instance type
# Be aware of the costs of these training instances. They will run as long as the training duration
# We use ml.c4.2xlarge or ml.c5.2xlarge in the AWS DeepRacer console


# instance_type = "ml.c4.2xlarge"
instance_type = "ml.p2.xlarge"


# Starting SageMaker session
sage_session = sagemaker.session.Session()

# Create unique job name.
job_name_prefix = 'deepracer-notebook'

# Duration of job in seconds (1 hours)
job_duration_in_seconds = 3600 * 1

# AWS Region
aws_region = sage_session.boto_region_name
if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia),"
                    "US West (Oregon) and EU (Ireland). Please switch to one of these regions.")

### Setup S3 bucket
Set up the linkage and authentication to the S3 bucket that we want to use for checkpoint and metadata.

#### **Note: This cell snapshots time to create folders** - training output will go to these folders. So running the code again means you are pointing to a different location. 

In [ ]:
# S3 bucket
s3_bucket = sage_session.default_bucket()

# SDK appends the job name and output folder
s3_output_path = 's3://{}/'.format(s3_bucket)

# Ensure that the S3 prefix contains the keyword 'sagemaker'
s3_prefix = job_name_prefix + "-sagemaker-" + strftime("%y%m%d-%H%M%S", gmtime())

# Get the AWS account id of this account
sts = boto3.client("sts")
account_id = sts.get_caller_identity()['Account']

print("Using s3 bucket {}".format(s3_bucket))
print("Model checkpoints and other metadata will be stored at: \ns3://{}/{}".format(s3_bucket, s3_prefix))

### Build and push docker image

The file ./Dockerfile contains all the packages that are installed into the docker. Instead of using the default sagemaker container. We will be using this docker container. 

If the docker file is not yet present, this takes about 8 minutes to complete. It takes a few seconds on subsequent runs.

In [ ]:
#!docker rm -f $(docker ps -a -q);
#!docker rmi -f $(docker images -q);

In [ ]:
%%time
from copy_to_sagemaker_container import get_sagemaker_docker, copy_to_sagemaker_container, get_custom_image_name
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
# repo name
repository_short_name = job_name_prefix + "-%s" % cpu_or_gpu
custom_image_name = get_custom_image_name(repository_short_name)

try:
    print("Copying files from your notebook to existing sagemaker container")
    sagemaker_docker_id = get_sagemaker_docker(repository_short_name)
    copy_to_sagemaker_container(sagemaker_docker_id, repository_short_name)
except Exception as e:
    print("Creating sagemaker container")
    docker_build_args = {
        'CPU_OR_GPU': cpu_or_gpu, 
        'AWS_REGION': boto3.Session().region_name,
    }
    custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
    print("Using ECR image %s" % custom_image_name)
    


### Train the RL model using the Python SDK Script mode

Next, we define the following algorithm metrics that we want to capture from cloudwatch logs to monitor the training progress. These are algorithm specific parameters and might change for different algorithm. We use [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html) for this example.

In [ ]:
metric_definitions = [
    # Training> Name=main_level/agent, Worker=0, Episode=19, Total reward=-102.88, Steps=19019, Training iteration=1
    {'Name': 'reward-training',
     'Regex': '^Training>.*Total reward=(.*?),'},
    
    # Policy training> Surrogate loss=-0.32664725184440613, KL divergence=7.255815035023261e-06, Entropy=2.83156156539917, training epoch=0, learning_rate=0.00025
    {'Name': 'ppo-surrogate-loss',
     'Regex': '^Policy training>.*Surrogate loss=(.*?),'},
     {'Name': 'ppo-entropy',
     'Regex': '^Policy training>.*Entropy=(.*?),'},
   
    # Testing> Name=main_level/agent, Worker=0, Episode=19, Total reward=1359.12, Steps=20015, Training iteration=2
    {'Name': 'reward-testing',
     'Regex': '^Testing>.*Total reward=(.*?),'},
]

We use the RLEstimator for training RL jobs.

1. Specify the source directory which has the environment file, preset and training code.
2. Specify the entry point as the training code
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container.
4. Define the training parameters such as the instance count, instance type, job name, s3_bucket and s3_prefix for storing model checkpoints and metadata. **Only 1 training instance is supported for now.**
4. Set the RLCOACH_PRESET as "deepracer" for this example.
5. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks.

In [ ]:
estimator = RLEstimator(entry_point="training_worker.py",
                        source_dir='src',
                        image_name=custom_image_name,
                        dependencies=["common/"],
                        role=sagemaker_role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        train_max_run=job_duration_in_seconds,
                        hyperparameters={
                            "s3_bucket": s3_bucket,
                            "s3_prefix": s3_prefix,
                            "aws_region": aws_region,
                            "preset_s3_key": "%s/presets/preset.py"% s3_prefix,
                            "model_metadata_s3_key": "%s/model_metadata.json" % s3_prefix,
                            "environment_s3_key": "%s/environments/deepracer_racetrack_env.py" % s3_prefix,
                        },
                        subnets=deepracer_subnets,
                        security_group_ids=deepracer_security_groups,
                    )

estimator.fit(wait=False)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

### Create the Kinesis video stream (optional)

In [ ]:
kvs_stream_name = "dr-kvs-{}".format(job_name)

!aws --region {aws_region} kinesisvideo create-stream --stream-name {kvs_stream_name} --media-type video/h264 --data-retention-in-hours 24
print ("Created kinesis video stream {}".format(kvs_stream_name))

### Start the Robomaker job

In [ ]:
robomaker = boto3.client("robomaker")

### Create Simulation Application

In [ ]:
robomaker_s3_key = 'robomaker/simulation_ws.tar.gz'
robomaker_source = {'s3Bucket': s3_bucket,
                    's3Key': robomaker_s3_key,
                    'architecture': "X86_64"}
simulation_software_suite={'name': 'Gazebo',
                           'version': '7'}
robot_software_suite={'name': 'ROS',
                      'version': 'Kinetic'}
rendering_engine={'name': 'OGRE',
                  'version': '1.x'}

### Upload your customized simulation application to your s3 bucket

The AWS DeepRacer bundle to be used by the AWS RoboMaker service is under `build/output.tar.gz`. Next, we need to upload the bundle to our S3 bucket and create an AWS RoboMaker Simulation Application.

In [ ]:
# Use your custom simApp
!aws s3 cp ./build/output.tar.gz s3://{s3_bucket}/{robomaker_s3_key}

### Create arn for the AWS RoboMaker simulation application

In [ ]:
# create arn for AWS RoboMaker
app_name = "deepracer-notebook-application" + strftime("%y%m%d-%H%M%S", gmtime())

print(app_name)
try:
    response = robomaker.create_simulation_application(name=app_name,
                                                       sources=[robomaker_source],
                                                       simulationSoftwareSuite=simulation_software_suite,
                                                       robotSoftwareSuite=robot_software_suite,
                                                       renderingEngine=rendering_engine)
    simulation_app_arn = response["arn"]
    print("Created a new simulation app with ARN:", simulation_app_arn)
except Exception as e:
    if "AccessDeniedException" in str(e):
        display(Markdown(generate_help_for_robomaker_all_permissions(role)))
        raise e
    else:
        raise e

## Section 4: Using multiple rollouts during training


We create AWS RoboMaker simulation jobs that simulates the environment and shares this data with SageMaker for training. Each roll-out uses a central model to independently collect experience in the form of episodes, where each episode consist of (state, action, next state, reward) tuples

![distribrollout](img/fourrollouts.png)


We use horizontal scaling where the neural network model files are synchronized between the Amazon Sagemaker training job and AWS RoboMaker simulation workers. Model sync behavior is coded in src/markov/training_worker.py 


![rollouts](./SageMaker_RoboMaker.png)


```
if graph_manager.agent_params.algorithm.distributed_coach_synchronization_type == 
            DistributedCoachSynchronizationType.SYNC:
    graph_manager.save_checkpoint()
else:
    graph_manager.occasionally_save_checkpoint()
```
To enable sync models to the s3 bucket for multiple rollouts, set the parameter in src/markov/presets/preset.py

```
agent_params.algorithm.distributed_coach_synchronization_type = DistributedCoachSynchronizationType.SYNC
```




### Objective: Please increase number of workers to 2

**Specify the number of roll-out workers** using the ***num_simulation_workers*** parameter.

In [ ]:
available_tracks = ["reinvent_base", # 0
                    "AWS_track", # 1
                    "Tokyo_Training_track", #2
                    "Virtual_May19_Train_track", #3 (london)
                    "reInvent2018_36inch", #4
                    "reInvent2018_mirror", #5
                    "reInvent2019_track"] #6

training_tracks_indices = [6, 6, 5, 5]

num_simulation_workers = len(training_tracks_indices)


In [ ]:

envriron_vars = {
    "WORLD_NAME": "reinvent_base",
    "KINESIS_VIDEO_STREAM_NAME": "SilverstoneStream",
    "PYTHONPATHS":"wohooo this worked",
    "ALTERNATE_DRIVING_DIRECTION": "false",
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "TRAINING_JOB_ARN": job_name,
    "APP_REGION": aws_region,
    "METRIC_NAME": "TrainingRewardScore",
    "METRIC_NAMESPACE": "AWSDeepRacer",
    "REWARD_FILE_S3_KEY": "%s/rewards/reward_function.py" % s3_prefix,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_prefix + "/training_metrics.json",
    "TARGET_REWARD_SCORE": "None",
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "deepracer_simulation_environment",
                                           "launchFile": "distributed_training.launch",
                                           "environmentVariables": envriron_vars}
                         }

vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    simulation_application["launchConfig"]["environmentVariables"]["WORLD_NAME"] = available_tracks[training_tracks_indices[job_no]]
    client_request_token = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    response =  robomaker.create_simulation_job(iamRole=sagemaker_role,
                                            clientRequestToken=client_request_token,
                                            maxJobDurationInSeconds=job_duration_in_seconds,
                                            failureBehavior="Fail",
                                            simulationApplications=[simulation_application],
                                            vpcConfig=vpcConfig
                                            )
    print(response)
    responses.append(response)


print("Created the following jobs:")
job_arns = [response["arn"] for response in responses]
for response in responses:
    print("Job ARN", response["arn"]) 
    
    

Copy the below output for log analysis and to determine the top performing models on the real world track.

In [ ]:
print("S3_MODEL_SAVE_PREFIX = '%s'" %experiment_name)

arn_str = list()
for response in responses:
    arn_str.append(response["arn"].split('/')[-1])
print('stream_name_list =' + str(arn_str))
print("sagemaker_simapp_name_list = ['" + str(job_name) + "']")

print("s3_bucket = '%s'" %s3_bucket)
print("s3_prefix = '%s'" %s3_prefix)
print('simulation_app_arn = "%s"' %simulation_app_arn)

### Visualizing the simulations in RoboMaker
You can visit the RoboMaker console to visualize the simulations or run the following cell to generate the hyperlinks.

In [ ]:
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

## Run evaluation in parallel

In [ ]:
sys.path.append("./src")

num_simulation_workers = 1


envriron_vars = {
    "WORLD_NAME": "reInvent2019_track",
    "KINESIS_VIDEO_STREAM_NAME": "SilverstoneStream",
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "MODEL_S3_BUCKET": s3_bucket,
    "PYTHONPATHS":"wohooo this worked",
    "MODEL_S3_PREFIX": s3_prefix,
    "ALTERNATE_DRIVING_DIRECTION": "false",
    "APP_REGION": aws_region,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_prefix + "/evaluation_metrics.json",
    "NUMBER_OF_TRIALS": "5", # Doesnt matter
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}

simulation_application = {
    "application":simulation_app_arn,
    "launchConfig": {
         "packageName": "deepracer_simulation_environment",
         "launchFile": "evaluation.launch",
         "environmentVariables": envriron_vars
    }
}
                            
vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses_eval = []
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                                outputLocation={ 
                                                  "s3Bucket": s3_bucket,
                                                  "s3Prefix": s3_prefix
                                                },
                                                maxJobDurationInSeconds=job_duration_in_seconds*3,
                                                iamRole=sagemaker_role,
                                                failureBehavior="Continue",
                                                simulationApplications=[simulation_application],
                                                vpcConfig=vpcConfig)
    responses_eval.append(response)

print('~~~ EVAL ~~~\n')
arn_str = list()
for response in responses_eval:
    arn_str.append(response["arn"].split('/')[-1])
print('eval_stream_name_list =' + str(arn_str))
    

## Section 5: How to analyze your models and determine top 5 models to test on the real track

Go to evaluation notebook?

### Clean up RoboMaker and SageMaker training job

Execute the cells below if you want to kill RoboMaker and SageMaker job.

In [ ]:
# # Cancelling robomaker job
# for job_arn in job_arns:
#     robomaker.cancel_simulation_job(job=job_arn)

# # Stopping sagemaker training job
# sage_session.sagemaker_client.stop_training_job(TrainingJobName=estimator._current_job_name)

### Clean Up Simulation Application Resource

In [ ]:
# robomaker.delete_simulation_application(application=simulation_app_arn)

### Clean your S3 bucket (Uncomment the awscli commands if you want to do it)

In [ ]:
## Uncomment if you only want to clean the s3 bucket
# sagemaker_s3_folder = "s3://{}/{}".format(s3_bucket, s3_prefix)
# !aws s3 rm --recursive {sagemaker_s3_folder}

# robomaker_s3_folder = "s3://{}/{}".format(s3_bucket, job_name)
# !aws s3 rm --recursive {robomaker_s3_folder}

# robomaker_sim_app = "s3://{}/{}".format(s3_bucket, 'robomaker')
# !aws s3 rm --recursive {robomaker_sim_app}

# model_output = "s3://{}/{}".format(s3_bucket, s3_bucket)
# !aws s3 rm --recursive {model_output}

### Clean the docker images
Remove this only when you want to completely remove the docker or clean up the space of the sagemaker instance

In [ ]:
# !docker rmi -f $(docker images -q)